In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import tensorflow as tf

from tensorflow.keras import layers

from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

from vit_for_tflite_compartible import create_vit_classifier, PatchEncoder

import numpy as np
# from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
    extract_patches_from_audio_dataset
)
from helper_functions_2 import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    )


In [18]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(train_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.
Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [19]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)

# x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
# x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
# x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [112]:
train_patches_ds = extract_patches_from_audio_dataset(train_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)
test_patches_ds = extract_patches_from_audio_dataset(test_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)
val_patches_ds = extract_patches_from_audio_dataset(val_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_patches_ds, data_type="not spectrogram")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(test_patches_ds, data_type="not spectrogram")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(val_patches_ds, data_type="not spectrogram")

In [76]:
for i in train_patches_ds:
    print(i)
    break

(<tf.Tensor: shape=(32, 23, 10, 64), dtype=float32, numpy=
array([[[[ 3.40763986e-01,  1.01113605e+00,  2.75454807e+00, ...,
          -2.74449047e-02, -9.71774578e-01, -8.15280676e-01],
         [-4.30774689e-01, -5.86046398e-01,  1.22522521e+00, ...,
           4.41522419e-01,  2.54840106e-01, -1.00589454e+00],
         [-1.57385692e-01,  1.86907768e-01,  5.69664121e-01, ...,
           4.63986814e-01,  2.59763861e+00,  1.47198403e+00],
         ...,
         [-1.78425217e+00, -2.02821183e+00, -2.26371789e+00, ...,
          -2.04176378e+00, -9.03327405e-01, -2.21873355e+00],
         [-2.89456391e+00, -2.56334162e+00, -3.09467173e+00, ...,
          -2.42361760e+00, -2.32472920e+00, -1.84095371e+00],
         [-3.63100529e+00, -3.30042052e+00, -3.12508631e+00, ...,
          -2.69808507e+00, -3.40792942e+00, -4.64528608e+00]],

        [[ 1.49402893e+00,  1.76572168e+00,  2.69736838e+00, ...,
           2.59917188e+00,  9.69423771e-01, -2.35847354e-01],
         [ 8.99882793e-01,  8

In [113]:
for i in x_val_np:
    print(i.shape)
    break

(23, 10, 64)


Attention mechnism is mechanism that allows each element of a sequence to focus on other elements of the same sequence.

queries (
𝑄
Q), keys (
𝐾
K), and values (
𝑉
V) understand this

In transformers, the MLP is part of the transformer block and is applied after the self-attention mechanism. It consists of two or more fully connected (dense) layers with a non-linear activation function applied between them. 

We give to model not image but patches

In [68]:
input_shape = (23, 10, 64)  
# input_shape = (46, 20, 16)  
num_classes = 2  
patch_size = 8
num_patches = (184 // patch_size) * (80 // patch_size) # 230
num_patches

230

In [117]:
input_shape = (23, 10, 64)
# input_shape = (46, 20, 16)  
num_classes = 2  
patch_size = 8
num_patches = (184 // patch_size) * (80 // patch_size) # 230

projection_dim = 8  # Size of embedding hpo # Less was tested - not successfull
num_heads = 4  # Number of attention heads hpo
transformer_layers = 1  # Number of transformer layers
mlp_head_units = [16, 8]  # Multi-Layer Perceptron head units
transformer_units = [projection_dim * 2, projection_dim]

vit_classifier = create_vit_classifier(input_shape, num_classes, patch_size, num_patches, projection_dim, num_heads, transformer_layers, transformer_units, mlp_head_units)
vit_classifier.summary()
vit_classifier.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_40 (InputLayer)       [(None, 23, 10, 64)]         0         []                            
                                                                                                  
 reshape_39 (Reshape)        (None, 230, 64)              0         ['input_40[0][0]']            
                                                                                                  
 dense_197 (Dense)           (None, 230, 8)               520       ['reshape_39[0][0]']          
                                                                                                  
 patch_encoder_36 (PatchEnc  (None, 230, 8)               1840      ['dense_197[0][0]']           
 oder)                                                                                     

In [118]:
vit_classifier.fit(train_patches_ds, epochs=20, validation_data=val_patches_ds)
print("Validation dataset accuracy:")
val_loss, val_acc = vit_classifier.evaluate(x_val_np, y_val_np)

Epoch 1/20
353/353 [==============================] - 35s 91ms/step - loss: 0.6135 - accuracy: 0.6676 - val_loss: 0.3027 - val_accuracy: 0.8993
Epoch 2/20
353/353 [==============================] - 32s 90ms/step - loss: 0.4136 - accuracy: 0.7673 - val_loss: 0.1892 - val_accuracy: 0.9326
Epoch 3/20
353/353 [==============================] - 32s 90ms/step - loss: 0.3249 - accuracy: 0.8402 - val_loss: 0.1535 - val_accuracy: 0.9486
Epoch 4/20
353/353 [==============================] - 31s 89ms/step - loss: 0.2817 - accuracy: 0.8683 - val_loss: 0.1366 - val_accuracy: 0.9493
Epoch 5/20
353/353 [==============================] - 31s 89ms/step - loss: 0.2515 - accuracy: 0.8843 - val_loss: 0.1371 - val_accuracy: 0.9609
Epoch 6/20
353/353 [==============================] - 32s 92ms/step - loss: 0.2154 - accuracy: 0.9031 - val_loss: 0.1044 - val_accuracy: 0.9616
Epoch 7/20
353/353 [==============================] - 31s 89ms/step - loss: 0.2087 - accuracy: 0.9131 - val_loss: 0.1095 - val_accuracy:

In [121]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(vit_classifier, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(vit_classifier, x_test_np, y_test_np, model_format="keras")

print("Time of one prediction for Test dataset:")
evaluate_time_of_prediction(vit_classifier, x_test_np, y_test_np, model_format="keras", show_prediction_evaluation=True)


Validation dataset:
44/44 [==============================] - 1s 33ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9669181198941834

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9700235926422601

Test dataset:
44/44 [==============================] - 1s 30ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.68%
Recall: 97.03%
Precision: 96.22%
F1-score: 96.62%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9662936091365731

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9670884651370979
Time of one prediction for Test dataset:
Accuracy: 97.68%
Recall: 97.03%


In [122]:
MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/hpo/vit_mel_spec_final.keras"
vit_classifier.save(MODEL_FILE_NAME)
print("Model file name: ", MODEL_FILE_NAME)
convert_bytes(get_file_size(MODEL_FILE_NAME), "MB")

Model file name:  ../spectrogram_models_from_notebooks/vit/hpo/vit_mel_spec_final.keras
File size: 0.482 Megabytes


In [123]:
model_loaded = tf.keras.models.load_model(MODEL_FILE_NAME, compile=True, custom_objects={ "PatchEncoder": PatchEncoder})


: 